<img src="http://drive.google.com/uc?export=view&id=1JzM1Jig5KAOCvU4tIf2t66B3gd1uy1rG" width=500px>

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.



## <font color='blue'> Table Of Contents </font> 

<font color='blue'>
    
- ### Introduction to Lab 2
- ### Database Setup
- ### Walkthrough of Receiver Simulator
- ### Use Cases
	* #### Internet of Vehicles platform for Volkswagen
## (Advanced)
- ### Device Authentication
	* #### Anonymous
	* #### Username and Passwords
	* #### Certificates
- ### Access Control
</font>

## <font color='blue'> Introduction to Lab 2 </font>


**Overview**


In this lab, you will work with the receiver simulator that collects the IoT device data and stores it in a MongoDB database.

We will also review some use cases of MQTT protocol in IoT systems.

In addition, we will look at authentication of devices, authorization to publish/subscribe in topics.

**Objectives**

After completing this lab, you will be able to:
- Fetch messages from MQTT broker and store it in a MongoDB database
- Review the use cases of MQTT broker in IoT systems
- Authenticate clients / devices
- Authorize the clients / devices to specific topics

**Duration**

This lab requires approximately **45 minutes** to complete.

---

## <font color='blue'> Database Setup </font>

Considering the semi-structured nature of data published by the IoT devices, it is generally preferable to use a NoSQL database. In this module, we will use one of the widely used NoSQL database engine, **MongoDB** (Community Edition).

You may follow the steps given at [https://docs.mongodb.com/manual/administration/install-community/](https://docs.mongodb.com/manual/administration/install-community/) to install the database engine on your computer. Instructions to start / stop the database engine is also described in the same link.

To ensure MongoDB is listening for connections, you can execute the following and confirm the response

**Windows**
```bash
netstat -an | findstr 27017
```
and you should see the response as follows
```
Active Connections

  Proto  Local Address          Foreign Address        State
  ...
  TCP    127.0.0.1:27017         DESKTOP-UTES7QV:0      LISTENING
  ...
```

**Linux**
```bash
netstat -ltn
```
and you should see the response as follows
```
Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State      
...
tcp        0      0 127.0.0.1:27017              127.0.0.1:*               LISTEN
...
```

**Mac**
```bash
lsof -nP -iTCP:27017 | grep LISTEN
```
and the response should be 
```
mongod  ... username    9u  IPv4 ...      0t0  TCP 127.0.0.1:27017 (LISTEN)
```

> The preceding `127.0.0.1` before `27017` means that the mongod service is listening to connections made to port 27017 only from the same computer.

---

## <font color='blue'> Walkthrough of Receiver Simulator </font>

Lab 1 introduced you to a Publish Simulator that keeps publishing MQTT messages as configured. 

This section of Lab 2 walks you through the Receiver Simulator that acts as an IoT client / device, receiving all the messages from the publishing devices / Publish Simulator and thus storing it on your MongoDB database.

The complete source code is available in the companion notebook ```M02-W02-IoT-Lab2-MQTT-Receiver-Source-Code.ipynb```

This is normally done by a central server (on-prem or cloud), either directly communicating with the devices, or via an Edge server.

In order to communicate with the MongoDB service from the python program, we will use the `pymongo` client library. You may install it by executing the following instruction on your Terminal

```bash
python -m pip install pymongo
```

**Step 1:** Create a python file in your computer  
> Example: ```subscribe.py```  

We will import the client submodule as mqtt and create an instance of the same

In [ ]:
import paho.mqtt.client as mqtt

#Instantiating an object with mqtt
client = mqtt.Client()

Defining the callback functions on `connect`, `disconnect`, `message`

In [ ]:
# Callback function - executed when the program successfully connects to the broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("devices/#")

#Callback function - executed when the program gracefully disconnects from the broker
def on_disconnect(client, userdata, rc):
    print("Disconnected with result code "+str(rc))

#Callback function - executed whenever a message is published to the topics that 
#this program is subscribed to
def on_message(client, userdata, msg):
    item = {"topic":msg.topic, "payload":msg.payload}
    dbt.insert_one(item)
    print("Received a messsage on " + msg.topic + " and inserted it to the DB")


#Setting callback functions for various client actions
client.on_connect = on_connect
client.on_message = on_message
client.on_disconnect = on_disconnect

Unlike the callback functions in Lab 1, you might realize a couple of changes here. 

In the `on_connect` function, we are subscribing to a partial multilevel wildcard topic **devices/#**. This will ensure all the topics from the simulator program in Lab 1 are subscribed to, in this program.

Also, for every message that the Lab 1 simulator program publishes, the `on_message` function in this program will insert the topic and the message into the MongoDB database.

Similar to the Lab 1, we are fetching the broker and database details from a configuration file `config.json`



```json
{
    "broker_host": "localhost",
    "broker_port": 1883,

    "db_host": "localhost",
    "db_port": 27017,
    "db_name": "iot-db",
    "db_collection":"iot-sensors-data"
}
```

Since the configuration is stored in json format, we import the contents of the file as json and further initialize the database connection

In [ ]:
import json
import time
import pymongo

#Reading the configuration file
f=open("config.json")
config = json.loads(f.read())
f.close()

#Initializing connection to the database
dbclient = pymongo.MongoClient(config["db_host"], config["db_port"])
db = dbclient[config["db_name"]]
dbt = db[config["db_collection"]]

In this lab tutorial, all the messages coming in the "devices/#" topic are stores in a single MongoDB database collection. 

However this may not be a preference in the production environment. A desired way of storing data would be to store the messages in a separate collection for a given topic or a category. There could be more structured collections derived from these dump/log collections for performing analytics.

We then connect the client to the broker from the broker configuration read from the `config.json` file and initialize the client loop which keeps checking for any client actions and executes the appropriate callback function

In [ ]:
#Connecting to broker
client.connect(host=config["broker_host"], port=config["broker_port"], keepalive=60)

'''
Start the MQTT client non-blocking loop to listen the broker for messages 
in subscribed topics and other operations for which the callback functions 
are defined
'''
client.loop_start()

The start of loop will take care of executing relevant callback functions to receive the message and insert it to the database. Hence the program is only expected to keep running until there is a Keyboard Interrupt, during which the client will be disconnected and the loop will be stopped.

This is achieved with the combination of an infinite while loop and an exception block to detect the Keyboard Interrupt

In [ ]:
while True:
    try:
        #Do nothing
        time.sleep(0.5)
    
    #Disconnect the client from MQTT broker and stop the loop gracefully at 
    # Keyboard interrupt (Ctrl+C)
    except KeyboardInterrupt:
        client.disconnect()
        client.loop_stop()
        break



**Step 2:** If you haven't done already, create the configuration file `config.json` in the same directory as that of your python program and copy-paste then save the following in it

```json
{
    "broker_host": "localhost",
    "broker_port": 1883,

    "db_host": "localhost",
    "db_port": 27017,
    "db_name": "iot-db",
    "db_collection":"iot-sensors-data"
}
```



**Step 3:** Start the python program

```bash
python subscribe.py
```

**Step 4:** Open another Terminal and navigate to the directory where you have stored `publish.py` from the Lab 1 and start the publish simulator

```shell
python publish.py
```

You should now see the relevant debug messages printed on the appropriate terminal windows.

**Step 5:** Let both the `publish` and `subscribe` simulators run for a few minutes so the database can collect some data and then stop the program `Ctrl+C`


**Optional:** You can view the data inserted into the database by performing a query as follows



In [ ]:
import pymongo
import json

#Reading the configuration file
f=open("config.json")
config = json.loads(f.read())
f.close()

#Initializing connection to the database
dbclient = pymongo.MongoClient(config["db_host"], config["db_port"])
db = dbclient[config["db_name"]]
dbt = db[config["db_collection"]]

#Querying for the messages that were published to the `devices/temp` topic
entries = dbt.find({"topic":"devices/temp"})

#Print the entries
for entry in entries:
    print(entry)

This program should display the entries inserted into the `db-collection` as follows

```json
{'_id': ObjectId('5ffe5db4715dd2b8e9e348ab'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:10:52", "value": 25.11, "device_type": "temperature", "device_id": "temperature_0"}'}
{'_id': ObjectId('5ffe5db5715dd2b8e9e348ac'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:10:52", "value": 24.02, "device_type": "temperature", "device_id": "temperature_1"}'}
{'_id': ObjectId('5ffe5db9715dd2b8e9e348ae'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:10:57", "value": 23.1, "device_type": "temperature", "device_id": "temperature_0"}'}
{'_id': ObjectId('5ffe5dba715dd2b8e9e348af'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:10:57", "value": 20.68, "device_type": "temperature", "device_id": "temperature_1"}'}
{'_id': ObjectId('5ffe5dbe715dd2b8e9e348b3'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:11:02", "value": 26.67, "device_type": "temperature", "device_id": "temperature_0"}'}
{'_id': ObjectId('5ffe5dbf715dd2b8e9e348b4'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:11:02", "value": 25.63, "device_type": "temperature", "device_id": "temperature_1"}'}
{'_id': ObjectId('5ffe5dc4715dd2b8e9e348b6'), 'topic': 'devices/temp', 'payload': b'{"timestamp": "2021-01-13 08:11:08", "value": 26.55, "device_type": "temperature", "device_id": "temperature_0"}'}
```

> Note: In the `subscribe.py` program, since we did not specify a unique id with which each object in the collection can be identified, MongoDB inserts it with the key `_id`

**Optional:** Modify `subscribe.py` to subscribe to the topics published by the co2 sensors and hence only those messages getting inserted into the database

---

## <font color='blue'> Device Authentication </font>

An MQTT can be configured to have any of the 3 types of authentications

### <font color='blue'> Anonymous </font>

This method allows any client/device with the valid hostname and the port number to get connected to the broker. 

To enable this authentication method, the following line can be added to `mosquitto.conf` file

```
allow_anonymous true
```
> Defaults to false, unless no listeners there are no listeners defined in the configuration file, in which case it is set to true, but connections are only allowed from the the local machine

<font color='red'>**Task 1:**</font> Since there weren't any listeners configured in the earlier steps, we were able to establish connection to the broker. Let's enforce the broker not to allow anonymous connections and find out the result

Edit the `mosquitto.conf` to have the following line
```
listener 1883 0.0.0.0
allow_anonymous false
```
Save and restart the broker

Now when we subscribe to a topic from the Terminal
```
mosquitto_sub -t "test"
```
the response we receive is
```
Connection error: Connection Refused: not authorised.
```
Modify the `mosquitto.conf` to allow anonymous connections, save and restart the broker. Then verify if the clients are able to establish connection to the broker

### <font color='blue'> Username and Password </font>
Allowing anonymous connections is obviously not a good choice for IoT systems in production. Authenticating the clients with a username and password is one of the authentication methods supported by mosquitto. The usernames and passwords (in encrypted format) is stored in a password file and the same is referenced in the `mosquitto.conf` file

To enable this authentication method with a password file, the `mosquitto.conf` needs to have the following lines in it
```
listener 1883 0.0.0.0
allow_anonymous false
password_file <passwordfilepath>
```
Say, if you choose to have the password file name as `pwfile`, to be stored in the same directory as that of default `mosquitto.conf` location, the **\<passwordfilepath\>** will be 

**Windows:**  _C:\Program Files\mosquitto\pwfile_

**Linux:** _/etc/mosquitto/pwfile_

**Mac:** _/usr/local/etc/mosquitto/pwfile_

<font color='red'>**Task 2:**</font> Initially, the password file you configured might not exist, so to create a password file and add a user to it, execute the following instruction

```bash
#Add User by creating a new passwordfile
mosquitto_passwd -c <passwordfilepath> <username>
```
> Replace \<username\> with a string of your choice, and enter the desired password for this username when prompted

To add more users in the same password file, you can use the following instruction
```bash
#Add a new user in the existing users list (password file)
mosquitto_passwd <passwordfilepath> <username>
#Enter the password when prompted
```

When you open the password file with a text editor, you will see something similar to the following
```
roger:$6$clQ4Ocu312S0qWgl$Cv2wUxgEN73c6C6jlBkswqR4AkHsvDLWvtEXZZ8NpsBLgP1WAo/qA+WXcmEN/mjDNgdUwcxRAveqNMs2xUVQYA==
sub_client:$6$U+qg0/32F0g2Fh+n$fBPSkq/rfNyEQ/TkEjRgwGTTVBpvNhKSyGShovH9KHewsvJ731tD5Zx26IHhR5RYCICt0L9qBW0/KK31UkCliw==
pub_client:$6$vxQ89y+7WrsnL2yn$fSPMmEZn9TSrC8s/jaPmxJ9NijWpkP2e7bMJLz78JXR1vW2x8+T3FZ23byJA6xs5Mt+LeOybAHwcUv0OCl40rA==
```

<font color='red'>**Task 3:**</font> Publish and subscribe with username and password

Once the `mosquitto.conf` file is modified as mentioned in Task 1 and restarted the broker to incorporate new configurations, then users are added to the password file as discussed in Task 2, connections to the broker can be established using the `-u` flag to add the username attribute and the `-P` flag to add the password attribute

Execute the following in Terminal 1
```bash
mosquitto_sub -u "<username>" -P "<password>" -t "greatlearning"
```
Execute the following in Terminal 2
```bash
mosquitto_pub -u "<username>" -P "<password>" -t "greatlearning" -m "publish message"
```
>Make sure to replace the `<username>` and `<password>` with the ones you used in Task 2

**Other features of _mosquitto_passwd_ command**
To delete/remove a user from the password file, you can use the following instruction
```bash
#Delete/Remove user
mosquitto_passwd -D <passwordfilepath> <username>
```

To add users to the password file by passing both the username and password as arguments, you can use the following instruction
```bash
mosquitto_passwd -b <passwordfilepath> <username> <password>
```
Say you have a list of usernames and passwords in plain text file for several users that are supposed to be added to the password file. In such case you can configure that file as a password file and update the passwords from text to hashed format

```bash
#Update a password file from text to hashed
mosquitto_passwd -U <passwordfilepath>
```

### Setting Username and Password in paho.mqtt.client

To set the username and password for the `paho.mqtt.client` in python program, add the following instruction after instantiating the broker object

```python
client.username_pw_set(username="xxx", password="yyy")
```

<font color='red'>**Task 4:**</font> Configure your MQTT broker to enable password based authentication, and modify the `publish.py` and `subscribe.py` to communicate with the broker using their designated authentication


### <font color='blue'> Certificates </font>
Just like how we have HTTPS, secure connection of HTTP, MQTTS  is the secure connection of MQTT. 

| Defualt Ports | Insecure | Secured (SSL/TLS) |
|--|--|--|
| HTTP | 80 | 443 |
| MQTT | 1883 | 8883 |

In MQTTS, an x509 certificate is used by the client and the broker to exchange the information in an encrypted format.

Obtaining and working with the SSL/TLS certificates works best in production environments where the broker host has a domain name associated with it.

You may find more information on configuring this [here](https://www.mosquitto.org/man/mosquitto-tls-7.html)


---

## <font color='blue'> Access Control </font>

In addition to restricting the devices via various authentication techniques as discussed above, it is also possible to limit the authenticated devices to publish/subscribe to certain topics.

The limitation on access to topics uses the concept of **Access Control List (ACL)**. Rules of access control is defined in a file (`aclfile`) and linked to the broker's configuration file (`mosquitto.conf`).  

> By default, there is not aclfile configured in the broker and hence there is no limitation on the topics

To enable access control list, add the following line to your `mosquitto.conf` file

```
acl_file <aclfilepath>
```

If you choose to have the acl file name as aclfile, to be stored in the same directory as that of default mosquitto.conf location, the `<aclfilepath>` will be

Windows: C:\Program Files\mosquitto\aclfile

Linux: /etc/mosquitto/aclfile

Mac: /usr/local/etc/mosquitto/aclfile

You may also find an example of aclfile in the directory where you had the `mosquitto.conf` by default

Rules in an **aclfile** are defined in 3 ways

1. General rules - Applies to all the devices
2. User specific rules - Applies to a specific user, identified with the username
3. Client specific rules - Applies to a specific device, identified by ClientID

Here is an example of an aclfile containing rules defined in all the 3 types

```
# This affects access control for clients with no username.
topic read $SYS/#

# This only affects clients with username "roger".
user roger
topic foo/bar

# This affects all clients.
pattern write $SYS/broker/connection/%c/state
```

### General rules

The genral rules are be defined either for a specific topic or a topic that contains a wildcard. The first word in a rule specifies that the rule is meant for a topic. 

The second word in a rule defines the access. It can be of the following 3 access levels

* read
* write
* readwrite
* deny

The third word in a rule is the topic

Example:
```
topic readwrite greatlearning/lab2
topic read greatlearning/#
```

### User Specific rules
The user specific rules are very similar to general rules with 2 additions.

1. The term `user` and the <username> is mentioned before the rule definition

2. The user specific rules can be applied for a pattern in addition to a specific topic
    
Example:
```
user bob
topic readwrite greatlearning
pattern readwrite greatlearning/%u/
```

> `%u` in the above rule will take up the value of username for the specified user
    
    
### Client Specific rules
    
The client specific rules are defined for a topic pattern which has the term `%c` in it, that gets replaced with the ClientID
    
Example:
```
pattern write greatlearning/%c
```
    
This rule simply means that devices with client ID `XXX` will only be able to publish messages to a topic `greatlearning/XXX`
    
    
However, all the 3 types of rules get consolidated and are applied for every message that gets published to the broker. Also, a change in **aclfile** does **not require restart** of the broker to take effect.